In [1]:
import csv
import itertools
import operator
import numpy as np
import matplotlib.pyplot as plt
import theano
import theano.tensor as T
import nltk
from utils import softmax

%matplotlib inline

### Language Modeling

Formulas, etc

### Training Data and Preprocessing

To train our language model we need text to learn from. Fortunately we don't need any labels because all we want to do is predict the next word given a sequence of previous words.

I opted to use 15,000 reddit commits that I downloaded from a [dataset available on Google's BigQuery](https://bigquery.cloud.google.com/table/fh-bigquery:reddit_comments.2015_08). So our model will be talking like reddit commenters!

Before we can use the data to train our Neural Network there are a couple of things we must do:

#### 1. Tokenizing Text

We want to predict on a per-word basis, so we must *tokenize* the sentences into words. You could just split each of the comments by spaces, but that wouldn't properly tokenize punctuation. The sentence "He left!"  should be 3 tokens, "He", "left" and "!". That's why we use [NLTK's](http://www.nltk.org/) `word_tokenize` method, which does that for us.

#### 2. Remove Infrequent words

Get rid of infrequent words. The reason for this is two-fold. First, having a huge vocabulary will make our model very slow to train (we'll talk about why that is later). Secondly, because we don't have a lot of contextual examples for these infrequent words we probably wouldn't be able to learn how to use them correctly anyway. That's quite similar to humans learn. To really understand the how to use a word appropriately you want to see it in different contexts. In the code below we limit our vocabulary to the `vocabulary_size` most common words. All words that we don't include in our vocabulary will be replaced by "UNKNOWN_TOKEN" in a sentence. For example, if we don't include the word "nonlinearities" in our vocabulary the sentence "nonlineraties are important in Neural Networks" would become "UNKNOWN_TOKEN are important in Neural Networks". The "UNKNOWN_TOKEN" word is always includes in our vocabulary and we predict it just like any other word.

#### 3. Prepend special start and end tokens

We prepend a special start token `SENTENCE_START` to each sentence, and similarly we append a special `SENTENCE_END` token to each sentence. This will help us when generating new sentences later on, because we can ask, given that the first word is "SENTENCE_END", what is the likely next word? And similarly, if our model predicts an "SENTENCE_END" token we know we should stop.

#### 4. Create word mappings

We create a mapping between words and indices, `index_to_word`, and `word_to_index`. For example, "network" may be index `301`.

#### 5. Build training data

For example, "he went home." could be `[999, 179, 341, 416, 0]`, where 999 maps to `SENTENCE_START` and 1000 maps to `SENTENCE_END` This is one training example, x. The corresponding y vector would be `[179, 341, 416, 9, 1000]`. It just the x vector shifted by one position and the last element is the `SENTENCE_END` token. Also, we don't actually want to represent the words in our x vectors as integers. Rather, we need to them to be one-hot vectors. A one-hot vector for 179 would be a vector of length `vocabulary_size` with all elements 0 except for the element at position 179, which is 1.


In [2]:
# We keep this many words
vocabulary_size = 2500
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

# Read the data and append SENTENCE_START and SENTENCE_END tokens
with open('data/reddit-comments-2015-08.csv', 'rb') as f:
    reader = csv.reader(f, skipinitialspace=True)
    reader.next()
    sentences = ["%s %s %s" % (sentence_start_token, x[0].decode('utf-8').lower(), sentence_end_token) for x in reader]

# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))

# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = dict(enumerate(x[0] for x in vocab))
index_to_word[vocabulary_size-1] = unknown_token
word_to_index = inv_map = {v: k for k, v in index_to_word.items()}

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

In [65]:
# Create the training data
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

In [39]:
# Print an training data example
x_example = X_train[8][:6]
y_example = y_train[8][:6]
print "X: '%s': %s" % (" ".join([index_to_word[x] for x in x_example]), x_example)
print "y: '%s': %s" % (" ".join([index_to_word[x] for x in y_example]), y_example)

X: 'SENTENCE_START i think you might be': [13, 4, 73, 8, 189, 26]
y: 'i think you might be missing': [4, 73, 8, 189, 26, 995]


### Building the RNN

TODO

First, let's review the equations for a RNN:

$
\begin{align}
s_t &= \tanh(Ux_t + Ws_{t_1}) \\
o_t &= \mathrm{softmax}(Vs_t)
\end{align}
$

In [5]:
class RNNNumpy:
    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Helper matrix to convert X into a one-hot vector
        self.to_onehot = np.eye(self.word_dim)
        # Randomly initialize the network parameters
        self.U = np.random.randn(hidden_dim, word_dim) * np.sqrt(2.0/word_dim)
        self.V = np.random.randn(word_dim, hidden_dim) * np.sqrt(2.0/hidden_dim)
        self.W = np.random.randn(hidden_dim, hidden_dim) * np.sqrt(2.0/hidden_dim)
        
    def forward_propagation(self, x):
        # The number of time steps
        x = self.to_onehot[x]
        T = len(x)
        # During forward propagation we save all hidden states in s. We need them later.
        # We add one additional element for the initial hidden, which we set to 0
        s = np.zeros((T + 1, self.hidden_dim))
        s[-1] = np.zeros(self.hidden_dim)
        # The outputs at each time step. Again, we save them for later.
        o = np.zeros((T, self.word_dim))
        # For each time step...
        for t in np.arange(T):
            s[t] = np.tanh(self.U.dot(x[t]) + self.W.dot(s[t-1]))
            o[t] = softmax(self.V.dot(s[t]))
        return [o, s]
        
    def predict(self, x):
        # Perform forward propagation and return index of the highest score
        o, s = self.forward_propagation(x)
        return np.argmax(o, axis=1)
    
    def calculate_loss(self, X, y):
        # We accumulate the total loss in L
        L = 0
        for i in np.arange(len(y)):
            x_i, y_i = X[i], y[i]
            o, s = self.forward_propagation(x_i)
            L += -1 * np.sum(np.log(o[np.arange(len(y_i)), y_i]))
        return L
    
    def calculate_mean_loss(self, X, y):
        # Divide calculate_loss by the number of words
        num_words = np.sum([len(y_i) for y_i in y])
        return self.calculate_loss(X,y)/float(num_words)   
            
    def bptt(self, x, y):
        T = len(y)
        # Perform forward propagation
        o, s = self.forward_propagation(x)
        # We accumulate the gradients in these variables
        dLdU = np.zeros(self.U.shape)
        dLdV = np.zeros(self.V.shape)
        dLdW = np.zeros(self.W.shape)
        delta_o = o
        delta_o[np.arange(len(y)), y] -= 1.
        # For each output backwards...
        for t in np.arange(T)[::-1]:
            dLdV += np.outer(delta_o[t], s[t].T)
            # Initial delta calculation
            delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
            # Backpropagation through time (for at most self.bptt_truncate steps)
            for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
                # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
                dLdW += np.outer(delta_t, s[bptt_step-1])
                dLdU += np.outer(delta_t, self.to_onehot[x[bptt_step]])
                # Update delta for next step
                delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
        return [dLdU, dLdV, dLdW]
    

In [103]:
def gradient_check(model, x, y, h=0.001, error_threshold=0.01):
    # Overwrite the bptt attribute. We need to backpropagate all the way to get the correct gradient
    model.bptt_truncate = 1000
    # Calculate the gradients using backprop
    bptt_gradients = model.bptt(x, y)
    # List of all parameters we want to chec.
    model_parameters = ['U', 'V', 'W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(model)
        print "Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            gradplus = model.calculate_loss([x],[y])
            parameter[ix] = original_value - h
            gradminus = model.calculate_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            parameter[ix] = original_value
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print "Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix)
                print "+h Loss: %f" % gradplus
                print "-h Loss: %f" % gradminus
                print "Estimated_gradient: %f" % estimated_gradient
                print "Backpropagation gradient: %f" % backprop_gradient
                print "Relative Error: %f" % relative_error
                return 
            it.iternext()
        print "Gradient check for parameter %s passed." % (pname)
            
            
np.random.seed(10)
# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 5
model = RNNNumpy(grad_check_vocab_size, 10)
gradient_check(model, [0,1,2,3], [1,2,3,4])

Performing gradient check for parameter U with size 50.
Gradient check for parameter U passed.
Performing gradient check for parameter V with size 50.
Gradient check for parameter V passed.
Performing gradient check for parameter W with size 100.
Gradient check for parameter W passed.


In [6]:
def train_rnn_with_sgd(model, X_train, y_train, learning_rate=0.0005, print_loss_after=1000, nepoch=1, anneal_after=-1, anneal_factor=0.5):
    # We keep track of the losses so we can plot them later
    losses = []
    for epoch in range(nepoch):
        for i in np.arange(len(y_train)):
            x_i, y_i = X_train[i], y_train[i]
            dLdU, dLdV, dLdW = model.bptt(x_i, y_i)
            model.U -= learning_rate * dLdU
            model.V -= learning_rate * dLdV
            model.W -= learning_rate * dLdW
            if (i % print_loss_after == 0):
                loss = model.calculate_mean_loss(X_train, y_train)
                losses.append(loss)
                print "Loss after epoch=%d i=%d: %f" % (epoch, i,loss)
        # Adjust the learning rate
        if(epoch % anneal_after == 0):
            learning_rate = learning_rate * anneal_factor


In [13]:
# np.random.seed(10)
# model = RNNNumpy(vocabulary_size, 100, bptt_truncate=4)
# losses = train_rnn_with_sgd(model, X_train_onehot[:100], y_train[:100], nepoch=100, learning_rate=0.005, anneal_after=5)

In [189]:
# How long would it take to train a model
np.random.seed(10)
model = RNNNumpy(vocabulary_size, 100, bptt_truncate=4)
# %timeit model.forward_propagation(X_train[3])
# %timeit model.bptt(X_train[3], y_train[3])
o, s = model.forward_propagation(X_train[3])
print o[1,:10]

[ 0.00042649  0.000451    0.00043382  0.00041876  0.00038081  0.00039459
  0.00036433  0.00038306  0.00038365  0.00039345]


In [82]:
class RNNTheano:
    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        U = np.random.randn(hidden_dim, word_dim) * np.sqrt(2.0/word_dim)
        V = np.random.randn(word_dim, hidden_dim) * np.sqrt(2.0/hidden_dim)
        W = np.random.randn(hidden_dim, hidden_dim) * np.sqrt(2.0/hidden_dim)
        # Theano: Created shared variables
        self.U = theano.shared(name='U', value=U.astype(theano.config.floatX))
        self.V = theano.shared(name='V', value=V.astype(theano.config.floatX))
        self.W = theano.shared(name='W', value=W.astype(theano.config.floatX))
        # We store the Theano graph here
        self.theano = {}
        self.__theano_build__()
    
    def __theano_build__(self):
        U, V, W = self.U, self.V, self.W
        x = T.ivector('x')
        y = T.ivector('y')
        x_onehot = T.extra_ops.to_one_hot(x, self.word_dim)
        def forward_prop_step(x_t, s_t_prev, U, V, W):
            s_t = T.tanh(U.dot(x_t) + W.dot(s_t_prev))
            o_t = T.nnet.softmax(V.dot(s_t))
            return [o_t[0], s_t]
        [o,s], updates = theano.scan(
            forward_prop_step,
            sequences=x_onehot,
            outputs_info=[None, dict(initial=T.zeros(self.hidden_dim))],
            non_sequences=[U, V, W],
            truncate_gradient=self.bptt_truncate,
            strict=True)
        
        prediction = T.argmax(o, axis=1)
        o_error = T.sum(T.nnet.categorical_crossentropy(o, y))
        
        # Gradients
        dU = T.grad(o_error, U)
        dV = T.grad(o_error, V)
        dW = T.grad(o_error, W)
        
        # Assign functions
        self.forward_propagation = theano.function([x], o)
        self.predict = theano.function([x], prediction)
        self.ce_error = theano.function([x, y], o_error)
        self.bptt = theano.function([x, y], [dU, dV, dW])
        
        # SGD
        learning_rate = T.scalar('learning_rate')
        self.sgd_step = theano.function([x,y,learning_rate], [], 
                      updates=[(self.U, self.U - learning_rate * dU),
                              (self.V, self.V - learning_rate * dV),
                              (self.W, self.W - learning_rate * dW)])
    
    def calculate_loss(self, X, Y):
        return np.sum([self.ce_error(x,y) for x,y in zip(X,Y)])
    
    def calculate_mean_loss(self, X, Y):
        # Divide calculate_loss by the number of words
        num_words = np.sum([len(y) for y in Y])
        return self.calculate_loss(X,Y)/float(num_words)   


In [83]:
def gradient_check_theano(model, x, y, h=0.001, error_threshold=0.01):
    # Overwrite the bptt attribute. We need to backpropagate all the way to get the correct gradient
    model.bptt_truncate = 1000
    # Calculate the gradients using backprop
    bptt_gradients = model.bptt(x, y)
    # List of all parameters we want to chec.
    model_parameters = ['U', 'V', 'W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter_T = operator.attrgetter(pname)(model)
        parameter = parameter_T.get_value()
        print "Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            parameter_T.set_value(parameter)
            gradplus = model.calculate_loss([x],[y])
            parameter[ix] = original_value - h
            parameter_T.set_value(parameter)
            gradminus = model.calculate_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            parameter[ix] = original_value
            parameter_T.set_value(parameter)
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print "Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix)
                print "+h Loss: %f" % gradplus
                print "-h Loss: %f" % gradminus
                print "Estimated_gradient: %f" % estimated_gradient
                print "Backpropagation gradient: %f" % backprop_gradient
                print "Relative Error: %f" % relative_error
                return 
            it.iternext()
        print "Gradient check for parameter %s passed." % (pname)
        
np.random.seed(10)
# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 5
model = RNNTheano(grad_check_vocab_size, 10)
gradient_check_theano(model, [0,1,2,3], [1,2,3,4])

Performing gradient check for parameter U with size 50.
Gradient check for parameter U passed.
Performing gradient check for parameter V with size 50.
Gradient check for parameter V passed.
Performing gradient check for parameter W with size 100.
Gradient check for parameter W passed.


In [12]:
np.random.seed(10)
model = RNNTheano(vocabulary_size, 100, bptt_truncate=4)
# model.forward_propagation(X_train[3])[1][:10]
# model.predict(X_train[3])
# model.ce_error(X_train[3], y_train[3])
# print X_train[:5]

In [15]:
%timeit model.calculate_mean_loss(X_train[:100], y_train[:100])

1 loops, best of 3: 4.04 s per loop


In [93]:
def train_rnn_with_sgd_theano(model, X_train, y_train, learning_rate=0.0005, print_loss_after=5000, nepoch=1, anneal_after=-1, anneal_factor=0.5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        for i in np.arange(len(y_train)):
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            if (num_examples_seen % print_loss_after == 0):
                loss = model.calculate_mean_loss(X_train, y_train)
                losses.append(loss)
                print "Loss after epoch=%d i=%d: %f" % (epoch, i,loss)
            num_examples_seen += 1
        # Adjust the learning rate
        if(epoch % anneal_after == 0):
            learning_rate = learning_rate * anneal_factor

In [ ]:
np.random.seed(10)
model = RNNTheano(vocabulary_size, 100, bptt_truncate=4)
losses = train_rnn_with_sgd_theano(model, X_train[:5000], y_train[:5000], nepoch=100, learning_rate=0.005, anneal_after=5, print_loss_after=15000)